## Final Project Submission

Please fill out:
* Student name: Linh Mai
* Student pace: self paced 
* Scheduled project review date/time: June 3rd, 2020
* Instructor name: Jeff Herman
* Blog post URL: https://linhmai19.github.io/

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline 
plt.style.use('seaborn')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.api as sms
import scipy.stats as stats

In [ ]:
df = pd.read_csv('cleaned_kc_house_data.csv')
df = df.drop('Unnamed: 0', 1)
df.head()

### Deal with categorical data

Check the scatter plots for of each feature against the target 'price'

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16,6))

for xcol, ax in zip(df.columns[1:4], axes):
    df.plot(kind='scatter', x=xcol, y='price', ax=ax, alpha=0.3, color='peru', sharey=True)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16,6))

for xcol, ax in zip(df.columns[4:7], axes):
    df.plot(kind='scatter', x=xcol, y='price', ax=ax, alpha=0.3, color='peru', sharey=True)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16,6))

for xcol, ax in zip(df.columns[7:10], axes):
    df.plot(kind='scatter', x=xcol, y='price', ax=ax, alpha=0.3, color='peru', sharey=True)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16,6))

for xcol, ax in zip(df.columns[10:13], axes):
    df.plot(kind='scatter', x=xcol, y='price', ax=ax, alpha=0.3, color='peru', sharey=True)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16,6))

for xcol, ax in zip(df.columns[13:16], axes):
    df.plot(kind='scatter', x=xcol, y='price', ax=ax, alpha=0.3, color='peru', sharey=True)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16,6))

for xcol, ax in zip(df.columns[16:19], axes):
    df.plot(kind='scatter', x=xcol, y='price', ax=ax, alpha=0.3, color='peru', sharey=True)

In [ ]:
for column in df.columns:
    print(column, df[column].nunique())

Based on the histograms and the number of unique values above, the categorical data are basements, bedrooms, bathrooms, floors, waterfront, view, condition, grade, yr_built, yr_sold, latitude, longitude, and zipcode. 

However, bathrooms, bedrooms, floors, grade variables have numeric data and hierarchical orders. For example, 3 bedrooms ranks higher than 2 bedrooms which ranks higher than 1 bedroom. Therefore, these features can be used in models without having to perform label encoding or creating dummy variables.

For the 'zipcode', it does not have a hierachical order. Therefore, one hot encoding is done in order to include them in the model. However, there are 70 unique zipcodes which are a lot to create addition columns. These 70 zipcodes are placed in 10 different bins and then performing one hot encoding.

In [ ]:
#checking the pentiles of zipcode
df['zipcode'].quantile([.1, .2, .3, .4, .5, .6, .7, .8, .9, 1])

#creating bins based on the quartiles
bins = [98000, 98008, 98028, 98038, 98053, 98065, 98103, 98115, 98125, 98155, 98200]
bins_cut = pd.cut(df['zipcode'], bins)
bins_cut = bins_cut.cat.as_ordered()
bins_cut.head()

In [ ]:
# Plot to check the created bins for 'zipcode'
graph = sns.countplot(data=df, x=bins_cut, order=bins_cut.value_counts().index)
graph.set_xticklabels(graph.get_xticklabels(), rotation=45)

#### Create dummy variables for 'zipcode' and deal with dummy variable trap

In [ ]:
# Now, replace the original 'zipcode' column with the created 'bins_cut'
df['zipcode']=bins_cut

# Create dummy variables, dropping one variable to avoid dummy variable trap
zipcode_dummies = pd.get_dummies(df['zipcode'], prefix = 'zip', drop_first=True)

# Then, drop the original 'zipcode' column
df = df.drop('zipcode', axis=1).copy()

# Add the zipcode dummy variables into the existing dataframe
df = pd.concat([df, zipcode_dummies], axis=1).copy()

df.head()

We also need to change the name of zipcode_dummies columns to avoid errors when doing modeling 

In [ ]:
df = df.rename(columns={'zip_(98008, 98028]':'zip_98008_98028', 'zip_(98028, 98038]':'zip_98028_98038',
                        'zip_(98038, 98053]':'zip_98038_98053', 'zip_(98053, 98065]':'zip_98053_98065', 
                        'zip_(98065, 98103]':'zip_98065_98103', 'zip_(98103, 98115]':'zip_98103_98115', 
                        'zip_(98115, 98125]':'zip_98115_98125', 'zip_(98125, 98155]':'zip_98125_98155',
                        'zip_(98155, 98200]':'zip_98155_98200'}).copy()

df.head()

The latitude and longitude are also categorical variables with no hierachical order. However, since the zipcode is already "treated", we can used this data to predict the house price for a simpler model. 

In [ ]:
# Drop both 'lat' and 'long' columns
df = df.drop(['lat', 'long'], axis=1).copy()

#### Create dummy variables for basement, condition, view, waterfront, yr_built, yr_sold

In [ ]:
# Create bins for yr_built
bins_yr_built = pd.cut(df['yr_built'], [1899, 1920, 1940, 1960, 1980, 2000, 2020])
bins_yr_built = bins_yr_built.cat.as_unordered()

# Create dummy variables, dropping one variable to avoid dummy variable trap
yr_built_dummies = pd.get_dummies(bins_yr_built, prefix='yr_built', drop_first=True)
yr_sold_dummies = pd.get_dummies(df['yr_sold'], prefix='yr_sold', drop_first=True)
view_dummies = pd.get_dummies(df['view'], prefix='view', drop_first=True)
wt_front_dummies = pd.get_dummies(df['waterfront'], prefix='wt_front', drop_first=True)
con_dummies = pd.get_dummies(df['condition'], prefix='con', drop_first=True)
base_dummies = pd.get_dummies(df['basement'], prefix='base', drop_first=True)

# Drop the original columns and add dummy variables into the existing dataframe
df = df.drop(['yr_built','yr_sold','view','waterfront','condition','basement'], axis=1).copy()
df = pd.concat([df, yr_built_dummies, yr_sold_dummies, view_dummies, wt_front_dummies, 
                con_dummies, base_dummies], axis=1).copy()

## Check for Multicollinearity of features

In [ ]:
fig = plt.figure(figsize=(12,8))
sns.heatmap(df.corr() > 0.75)
plt.title('Correlation > 0.75')

From the heatmap plot above, 'sqft_living' is correlated with 'sqft_above' and 'sqft_lot' is correlated with 'sqft_lot15'.

Plot 'sqft_living', 'sqft_above', 'sqft_lot' and 'sqft_lot15' against the target 'price' to determine which features are highly correlated with the target higher than others. 

In [ ]:
fig2 = plt.figure(figsize=(20,20))
sns.heatmap(df.corr(), annot=True)

Correlation values between: 

Price and 'sqft_living' = 0.53

Price and 'sqft_above' = 0.4

Price and 'sqft_lot' = -0.076

Price and 'sqft_lot15' = -0.085

Based on the correlation values above, we drop the 'sqft_above' and 'sqft_lot15' columns

In [ ]:
df = df.drop(['sqft_above', 'sqft_lot15'], axis=1).copy()

## Normalizing the data

We need to convert all of our numeric columns to the same scale by normalizing our dataset. We normalize the data by converting each numeric value to it's corresponding z-score for the column, which is obtained by subtracting the column's mean and then dividing by the column's standard deviation for every value.

'price' is not normalized because this is the feature to be predicted.

In [ ]:
df.info()

In [ ]:
#Use KDE plots to visualize the distribution of variables 
for feat in ['sqft_living', 'sqft_lot', 'sqft_living15', 'floors']:
    sns.kdeplot(df[feat], bw=0.5)
    plt.figure()

In [ ]:
for feat in ['sqft_living', 'sqft_lot', 'sqft_living15', 'floors']:
    df[feat] = df[feat].map(lambda x: np.log(x))
df.describe()

Checking the KDE plots again for the results after normalizing

In [ ]:
for feat in ['sqft_living', 'sqft_lot', 'sqft_living15', 'floors']:
    sns.kdeplot(df[feat], bw=0.5)
    plt.figure()

## Scaling the data

Now that the independent variables follow a normal distribution. The next step is equalize their magnitudes by scaling them using min-max scaling method. Only continuous variables are selected for this process

In [ ]:
# Define a function for min-max scaling
def scale(column):
    maximum = column.max()
    minimum = column.min()
    y = (column - minimum)/(maximum-minimum)
    return y

scale_vars = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'view', 'condition', 'grade', 'sqft_living15']

for feat in scale_vars:
    df[feat] = scale(df[feat])

In [ ]:
# Check to see whether the features have been scaled
df[scale_vars].describe()